# Vanilla LSTM for Sentiment Classification

This LSTM performs sentiment analysis on the IMDB review dataset.

In [ ]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten, Dropout, Activation, SpatialDropout1D
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import os
import sklearn.metrics
from sklearn.metrics import roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### Set Hyperparameters

In [ ]:
output_dir = 'model_output/vanillaLSTM'

e_param = 0.006
epochs = 10
batch_size = 64
patience = 2
val_split = .2

n_dim = 256
n_unique_words = 20000
max_review_length = 300
pad_type = trunc_type = 'pre'

n_lstm = 128 
drop_lstm = 0.5

#### Load Data

In [ ]:
(X_train, y_train), (X_valid, y_valid) = imdb.load_data(num_words=n_unique_words)

#### Preprocess Data

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
X_valid = pad_sequences(X_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

#### Design Deep Net Architecture

In [ ]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length))
model.add(LSTM(n_lstm, dropout=drop_lstm))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

#### Configure the Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
modelCheckpoint = ModelCheckpoint(filepath=output_dir + '/best-imdb-lstm-model.hdf5', save_best_only=True, mode='min')
earlyStopping = EarlyStopping(mode='min', patience=patience)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### TensorBoard

In [ ]:
tensorboard = TensorBoard("../logs/imdb-lstm-relus")

### Train the Model

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=val_split, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

#### Evaluate

In [ ]:
model = keras.models.load_model(output_dir + '/best-imdb-lstm-model.hdf5')

In [ ]:
y_hat = model.predict_proba(X_valid)

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat) * 100

In [ ]:
'{:0.2f}'.format(pct_auc)

In [ ]:
fpr, tpr, _ = sklearn.metrics.roc_curve(y_valid, y_hat)
roc_auc = sklearn.metrics.auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

plt.show()